# Optimal Power flow

#### Parameter Definitions

In our data file, we have lots of values, and multiple sheets. Take a moment to make sure you understand what every parameter means, conceptually. Below you can find the relevant parameter definitions.

##### Node (i.e. Bus) Data
**Parameter Name** | **Description**
---|---
Vmax        | Maximal voltage of node
Vmin        | Minimal voltage of node
VAnglemax   | Maximal voltage angle of node
VAnglemin   | Minimal voltage angle of node
Pload       | Real power drawn by loads at node
Qload       | Reactive power drawn by loads at node
PGmax       | Maximal real power of generators at node
PGmin       | Minimal real power of generators at node
QGmax       | Maximal reactive power of generators at node
QGmin       | Minimal reactive power of generators at node
a           | Generator cost function first coefficient# a*p^2
b           | Generator cost function second coefficient# b*p
c           | Generator cost function third coefficient# c

where the cost function of a dispatchable generator at node $i$, with a generation power of $p_i$, is defined as

$$ C_i(p_i) = a_i p_i^2 + b_i p_i + c_i $$

subject to the power constraints $ P^{G,min}_i \le p_i \le P^{Gmax}_i $.

**Note**: The slack node, i.e. the reference node, serves as an angular reference for all other nodes in the system, and is thus set to 0 deg. At this slack node, the voltage magnitude is also assumed to be 1.0 p.u.

##### Admittance Matrices

To model the admittance between two buses, i and k, we have set up the following matrices.
One matrix is for the real part of the admittance, while the other is for the imaginary part.
Since these matrices cover all relationships between buses, these are square matrices.
An example for the structure of the Real Admittance Matrix is shown below:

$$\begin{bmatrix}
G_{1,1} & G_{1,2} & \dots & \dots & G_{1,n} \\
G_{2,1} & \ddots & & & \vdots \\
\vdots & & G_{i,k} & & \vdots \\
\vdots & & & \ddots & G_{n-1,n} \\
G_{n,1} & \dots & \dots & G_{n,n-1} & G_{n,n} 
\end{bmatrix}$$

**Knowledge Checkup:**
To make sure you understand the data structure, take a look at the data file(s) and see if you can answer the following conceptual questions:

1) How many buses are there in total?
2) Which buses have generators connected to them?
3) Which buses have loads connected to them?
4) Which one of these nodes is the slack node?

After you have answered these questions for yourself, you're all ready to code!

#### Dependency Setup

In [ ]:
import os
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from opf_utils import solve_and_print, create_results_json

# TODO: Your email address (required for NEOS)
os.environ['NEOS_EMAIL'] = '...@tum.de'
FILENAME = 'data\\OPFData.xlsx'

#### Model Setup

In [ ]:
# filename = filename of the data source
# m = model
def add_node_data(filename, m):
    # TODO: Load the pandas dataframes for the node data.
    # Hint: Pay attention to the file types and headers. 
    data = None

    # TODO: Set the number of nodes
    m.nodes = None
    
    # TODO: Complete this function
    m.V_max = None
    m.V_min = None
    m.V_angle_max = None
    m.V_angle_min = None
    m.P_load = None
    m.Q_load = None
    m.P_G_max = None
    m.P_G_min = None
    m.Q_G_max = None
    m.Q_G_min = None
    m.a = None
    m.b = None
    m.c = None

    return m

def add_real_admittance_data(filename, m):
    # TODO: Load the pandas dataframes for the node data.
    # Hint: Pay attention to the file types and headers.
    data = None
    assert data.shape[0] == data.shape[1], "The admittance data is not a square matrix"

    # TODO: Complete this function
    m.G = None
    return m

def add_imag_admittance_data(filename, m):
    # TODO: Load the pandas dataframes for the node data.
    # Hint: Pay attention to the file types and headers.
    data = None
    assert data.shape[0] == data.shape[1], "The admittance data is not a square matrix"

    # TODO: Complete this function
    m.B = None
    return m

#### AC Optimal Power Flow Setup

In [ ]:
def objective_rule(m):
    # TODO: Complete this function
    return 0 # Eq. 1.1

# m = model
# i = node index
def power_bounds_rule(m, i):
    # TODO: Complete this function
    return (0,0,0) # Eq. 1.4

def voltage_angle_bounds_rule(m, i):
    # TODO: Complete this function
    return (0,0,0) # Eq. 1.7

def reactive_power_bounds_rule(m, i):
    # TODO: Complete this function
    return (0,0,0) # Eq. 1.5

def voltage_bounds_rule(m, i):
    # TODO: Complete this function
    return (0,0,0) # Eq. 1.6

def power_balance_rule_ac(m, i):
    # TODO: Complete this function
    return 0 == 0 # Eq. 1.2

def reactive_power_balance_rule(m, i):
    # TODO: Complete this function
    return 0 == 0 # Eq. 1.3

def add_linear_constraints_ac(m):
    # TODO: Complete this function
    m.power_bounds = None # Using power_bounds_rule
    m.voltage_angle_bounds = None # Using voltage_angle_bounds_rule
    m.reactive_power_bounds = None # Using reactive_power_bounds_rule
    m.voltage_bounds = None # Using voltage_bounds_rule
    return m

def add_nonlinear_constraints_ac(m):
    m.power_balance = None # Using power_balance_rule_ac
    m.reactive_power_balance = None # Using reactive_power_balance_rule
    return m

##### DC Optimal Power Flow Setup

In [ ]:
# m = model
# i = node index
def power_balance_rule_dc(m, i):
    # TODO: Complete this function, similar to power_balance_rule_ac
    return 0 == 0 # Eq. 1.12

def add_linear_constraints_dc(m):
    # TODO: Complete this function, similar to add_linear_constraints_ac
    m.power_bounds = None # Eq. 1.13
    m.voltage_angle_bounds = None # Eq. 1.14
    return m

def add_nonlinear_constraints_dc(m):
    # TODO: Complete this function
    m.power_balance = None # Using power_balance_rule_dc
    return m

##### Load Model from Data

In [ ]:
# Loads the data from the given file into the model, defines the optimization problem, and returns the model at the end.
def load_model(filename, m, AC=True):
    # Fill the model with the parameters from the files
    m = add_node_data(filename, m)
    m = add_real_admittance_data(filename, m)
    m = add_imag_admittance_data(filename, m)

    # Define optimization variables
    m.x_p = pyo.Var(m.nodes,
                    within=pyo.Reals,
                    doc='Optimization variable (power generation)')
    m.x_q = pyo.Var(m.nodes,
                    within=pyo.Reals,
                    doc='Optimization variable (reactive power generation)')
    m.x_v = pyo.Var(m.nodes,
                    within=pyo.Reals,
                    doc='Optimization variable (voltage)')
    m.x_v_angle = pyo.Var(m.nodes,
                         within=pyo.Reals,
                         doc='Optimization variable (voltage angle)')
    
    # Define objective function
    m.objective_function = pyo.Objective(
        rule=objective_rule,
        sense=pyo.minimize,
        doc='minimize(cost = sum of all generator operating costs)')
    
    # Define the linear constraints
    if AC:
        m = add_linear_constraints_ac(m)
    else:
        m = add_linear_constraints_dc(m)
    
    # Define the nonlinear constraints
    if AC:
        m = add_nonlinear_constraints_ac(m)
    else:
        m = add_nonlinear_constraints_dc(m)

    return m

##### Solve AC Model

In [ ]:
AC = True
model = load_model(FILENAME, pyo.ConcreteModel(), AC)
solve_and_print(model, AC)
# TODO: Uncomment if you want to write the result to json file
# team_name = '' # Your team name, ex. 'Group_A'
# create_results_json(model, team_name, AC)

##### Solve DC Model

In [ ]:
AC = False
model = load_model(FILENAME, pyo.ConcreteModel(), AC)
solve_and_print(model, AC)
# TODO: Uncomment if you want to write the result to json file
# team_name = '' # Your team name, ex. 'Group_A'
# create_results_json(model, team_name, AC)